In [1]:
# Libraries and utilities

# RENINDER: FARE FIT.TRANSFORM SOLO SU TRAINING E DOPO SUL TEST SOLO .TRANSFORM
# REMINDER: NORMALIZZARE PRIMA DI FARE TUTTO IL PREPROCESSING 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold, RandomizedSearchCV,GridSearchCV, RepeatedKFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)

In [2]:
df_train=pd.read_csv("./Dataset/RavdessAudioOnlyFeatures_TRAIN.csv")

In [3]:
print(f"df_train shape = {df_train.shape}")

df_train shape = (1828, 434)


In [4]:
"""
c = df_train.columns[df_train.eq(df_train.iloc[0]).all()].tolist()
print("These columns has a constant value:\n")
print (c,len(c))

df_train=df_train.drop(c,axis=1)
new_c=df_train.columns[df_train.eq(df_train.iloc[0]).all()].tolist()
print("These columns has a constant value:\n")
print (new_c,len(new_c))

"""

'\nc = df_train.columns[df_train.eq(df_train.iloc[0]).all()].tolist()\nprint("These columns has a constant value:\n")\nprint (c,len(c))\n\ndf_train=df_train.drop(c,axis=1)\nnew_c=df_train.columns[df_train.eq(df_train.iloc[0]).all()].tolist()\nprint("These columns has a constant value:\n")\nprint (new_c,len(new_c))\n\n'

In [5]:
# Select categorical columns

df_train_copy=df_train
c=df_train_copy.select_dtypes(include=['object']).columns.tolist()

# Label-encoding for categorical columns

le=preprocessing.LabelEncoder()
for i in c:
    df_train_copy[i]=le.fit_transform(df_train_copy[i])

# Normalization for pure numerical features

df_normalized = df_train_copy.copy()
numeric_features = [n for n in df_normalized.columns if n!="emotion" and n!= "vocal_channel" and n!= "emotional_intensity" and n!= "statement" and n!= "repetition" and n!= "sex" and n!="filename" and n!="actor"]
scaler = RobustScaler()
scaled_features = scaler.fit_transform(df_normalized[numeric_features])
df_normalized[numeric_features] = scaled_features
df_normalized.head()

,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,filename,frame_count,...,stft_max_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_q99_w4,stft_kur_w4,stft_skew_w4
0,0.0,1,5,0,1,0,1,1,0,-0.703721,...,0.0,-0.402429,1.421380,0.948188,0.831973,0.536448,0.0,0.0,10.826275,-2.710136
1,0.0,1,5,0,1,1,1,1,18,-0.666674,...,0.0,1.702463,1.092210,0.597843,0.441189,0.253557,0.0,0.0,0.064887,0.425032
2,0.0,1,5,0,0,0,1,1,36,-0.740767,...,0.0,-0.402429,0.899688,0.680414,0.652634,0.569396,0.0,0.0,3.519767,-1.290314
3,0.0,1,5,0,0,1,1,1,54,-0.851860,...,0.0,1.346476,0.835496,0.532143,0.515441,0.405521,0.0,0.0,0.218601,-0.015380
4,0.0,1,1,0,1,0,1,1,72,-0.444465,...,0.0,1.018057,0.935801,0.644804,0.549313,0.296205,0.0,0.0,1.260594,-0.448574


In [6]:
col=[x for x in df_normalized.columns if x!="emotion"]
X=df_normalized[col].values
y = np.array(df_normalized["emotion"])

#holdout
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=0
)

In [7]:
np.unique(y_train, return_counts=True), np.unique(y_test, return_counts=True)

((array([0, 1, 2, 3, 4, 5, 6, 7]),
  array([196, 196, 101, 196, 196,  98, 196, 100], dtype=int64)),
 (array([0, 1, 2, 3, 4, 5, 6, 7]),
  array([84, 84, 43, 84, 84, 42, 84, 44], dtype=int64)))

KNN Tuning

In [7]:
param_grid = {
    "n_neighbors": np.arange(2,30),
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "cityblock","minkowski"], # cityblock is the manhattan distance
}

grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=param_grid,
    cv=RepeatedStratifiedKFold(random_state=0), # with no parameters it is with 5 split and 10 repetitions
    refit=True,
    scoring="f1_macro"
)

grid.fit(X_train, y_train)
clf = grid.best_estimator_

In [8]:
print(grid.best_params_)

{'metric': 'euclidean', 'n_neighbors': 14, 'weights': 'distance'}


In [9]:
learner = KNeighborsClassifier(n_neighbors= 14, metric= 'euclidean', weights= 'distance')
classifier = learner.fit(X_train, y_train)
predictions = classifier.predict(X_test)

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96        84
           1       0.99      0.95      0.97        84
           2       1.00      0.98      0.99        43
           3       0.99      0.99      0.99        84
           4       0.98      0.95      0.96        84
           5       0.95      0.98      0.96        42
           6       0.95      0.96      0.96        84
           7       0.96      1.00      0.98        44

    accuracy                           0.97       549
   macro avg       0.97      0.97      0.97       549
weighted avg       0.97      0.97      0.97       549



DT Tuning

In [12]:
param_grid = {'max_depth': [None, 2, 5, 10, 15, 20],
              'min_samples_split': [2, 5, 10, 15, 20],
              'min_samples_leaf': [1, 5, 10, 15, 20]
}

grid = GridSearchCV(
    DecisionTreeClassifier(),
    param_grid=param_grid,
    cv=RepeatedStratifiedKFold(random_state=0), # with no parameters it is with 5 split and 10 repetitions
    refit=True,
    scoring="f1_macro"
)

grid.fit(X_train, y_train)
clf = grid.best_estimator_

In [13]:
print(grid.best_params_)

{'max_depth': 20, 'min_samples_leaf': 15, 'min_samples_split': 2}


In [10]:
learner = DecisionTreeClassifier(max_depth=20,min_samples_split=2,min_samples_leaf=15)
classifier = learner.fit(X_train, y_train)
predictions = classifier.predict(X_test)

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98        84
           1       1.00      0.98      0.99        84
           2       1.00      1.00      1.00        43
           3       0.99      0.99      0.99        84
           4       1.00      1.00      1.00        84
           5       0.95      1.00      0.98        42
           6       1.00      0.99      0.99        84
           7       1.00      1.00      1.00        44

    accuracy                           0.99       549
   macro avg       0.99      0.99      0.99       549
weighted avg       0.99      0.99      0.99       549



In [11]:
# Now read the test_dataset and perform same preprocessing as train_dataset


df_test=pd.read_csv("./Dataset/RavdessAudioOnlyFeatures_TEST.csv")
#c_new = df_test.columns[df_test.eq(df_test.iloc[0]).all()].tolist() #constant columns
#df_test=df_test.drop(c_new,axis=1)#remove constant columns
df_test_copy=df_test
c1=df_test_copy.select_dtypes(include=['object']).columns.tolist()
le=preprocessing.LabelEncoder() #label-encoding categorical features
for i in c1:
    df_test_copy[i]=le.fit_transform(df_test_copy[i])

# Normalization for pure numerical features for test_dataset

df_test_normalized = df_test_copy.copy()
numeric_features = [n for n in df_test_normalized.columns if n!="emotion" and n!= "vocal_channel" and n!= "emotional_intensity" and n!= "statement" and n!= "repetition" and n!= "sex" and n!="filename" and n!="actor"]
scaler = RobustScaler()
#scaled_features = scaler.transform(df_normalized[numeric_features])
scaled_features = scaler.fit_transform(df_test_normalized[numeric_features]) 
df_test_normalized[numeric_features] = scaled_features


col=[x for x in df_test_normalized.columns if x!="emotion"]
X_new=df_test_normalized[col].values
y_new = np.array(df_test_normalized["emotion"])



In [13]:
# Now train classifier with the entire train_dataset and test it on the test_dataset

learner = DecisionTreeClassifier(max_depth=20,min_samples_split=2,min_samples_leaf=15)
classifier = learner.fit(X, y)
predictions = classifier.predict(X_new)
print(classification_report(y_new,predictions))


              precision    recall  f1-score   support

           0       0.40      0.50      0.44        96
           1       0.00      0.00      0.00        96
           2       0.00      0.00      0.00        48
           3       0.00      0.00      0.00        96
           4       0.00      0.00      0.00        96
           5       0.33      0.50      0.40        48
           6       0.17      0.25      0.20        96
           7       0.00      0.00      0.00        48

    accuracy                           0.15       624
   macro avg       0.11      0.16      0.13       624
weighted avg       0.11      0.15      0.13       624



d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
classifier.score(X_new,y_new)

0.15384615384615385